In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
!pip install optuna

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 19.9 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import joblib
from scipy.sparse import hstack, csr_matrix
from transformers import pipeline
from datasets import Dataset
import optuna
from sklearn.model_selection import cross_val_score
import time
import numpy as np

In [ ]:
# Load dataset
file_path = "cleaned_advanced_dataset_with_5k_sarcasm.csv"
data = pd.read_csv(file_path)

In [ ]:
# Load the sarcasm detection model
sarcasm_detector = pipeline("text-classification", model="dnzblgn/Sarcasm-Detection-Customer-Reviews", device=-1)


Device set to use cpu


In [ ]:
# Features (Text) and Target (Sentiment)
X = data['cleaned_text']
y = data['Sentiment']

In [ ]:
# Convert data to Hugging Face Dataset for batch processing
dataset = Dataset.from_pandas(pd.DataFrame({'text': X}))

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
# Process sarcasm detection in batches for GPU efficiency
results = sarcasm_detector(dataset['text'], batch_size=2)  # Adjust batch size based on memory


In [ ]:
# Extract labels and map them
label_mapping = {'LABEL_0': 0, 'LABEL_1': 1}  # 0 = nonsarcasm, 1 = sarcasm
data['Sarcasm_Flag'] = [label_mapping[result['label']] for result in results]


In [ ]:
# Split data
X_train, X_test, y_train, y_test, sarcasm_train, sarcasm_test = train_test_split(
    X, y, data['Sarcasm_Flag'], test_size=0.1, random_state=42
)


In [ ]:
# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True,smooth_idf=True , max_features=28000, ngram_range=(1, 3), max_df=0.90, min_df=3)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [ ]:
# Convert sarcasm flag to sparse matrix
sarcasm_train_sparse = csr_matrix(sarcasm_train.values.reshape(-1, 1))
sarcasm_test_sparse = csr_matrix(sarcasm_test.values.reshape(-1, 1))


In [ ]:
# Combine TF-IDF features with sarcasm flag
X_train_combined = hstack([X_train_tfidf, sarcasm_train_sparse])
X_test_combined = hstack([X_test_tfidf, sarcasm_test_sparse])

In [ ]:
from optuna.pruners import MedianPruner


# Number of trials
n_trials = 10

# Ensure training data exists
if X_train_combined is None or y_train is None:
    raise ValueError("❌ Error: Training data is missing!")

def objective(trial):
    print(f"\n🔵 Starting Trial {trial.number + 1}/{n_trials}")

    # **Optimized Search Space**
    C_svm = trial.suggest_loguniform("C_svm", 1e-2, 1e1)   # More compact range
    C_lr = trial.suggest_loguniform("C_lr", 1e-2, 1)       # Keeps it reasonable
    alpha_nb = trial.suggest_float("alpha_nb", 1e-2, 1, log=True)

    print(f"🔹 Hyperparams - C_svm: {C_svm:.3f}, C_lr: {C_lr:.3f}, alpha_nb: {alpha_nb:.3f}")

    # Define models
    clf_svm = SVC(C=C_svm, kernel="rbf", probability=True)
    clf_lr = LogisticRegression(C=C_lr, solver="liblinear")
    clf_nb = MultinomialNB(alpha=alpha_nb)

    # Voting Classifier
    voting_clf = VotingClassifier(
        estimators=[("svm", clf_svm), ("lr", clf_lr), ("nb", clf_nb)],
        voting="soft"
    )

    # **Faster Cross-Validation (cv=2)**
    scores = cross_val_score(voting_clf, X_train_combined, y_train, cv=2, scoring="accuracy", n_jobs=-1)

    mean_acc = scores.mean()
    print(f"✅ Trial {trial.number + 1}/{n_trials} - Accuracy: {mean_acc:.4f}")

    return mean_acc

# **Aggressive Pruning for Speed**
study = optuna.create_study(direction="maximize", pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1))
study.optimize(objective, n_trials=n_trials)

# Show best results
print("\n🎯 Best Trial Found:")
print(f"⭐ Accuracy: {study.best_trial.value:.4f}")
print(f"🏆 Best Hyperparameters: {study.best_trial.params}")


[I 2025-04-01 14:32:54,462] A new study created in memory with name: no-name-f9090098-1aeb-4663-ab97-d0adb62f2b53
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1744981985.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-2, 1e1)   # More compact range
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1744981985.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-2, 1)       # Keeps it reasonable



🔵 Starting Trial 1/10
🔹 Hyperparams - C_svm: 0.043, C_lr: 0.017, alpha_nb: 0.054


[I 2025-04-01 14:50:20,428] Trial 0 finished with value: 0.720286014300715 and parameters: {'C_svm': 0.04267477133241278, 'C_lr': 0.01718157564076153, 'alpha_nb': 0.053701245506332276}. Best is trial 0 with value: 0.720286014300715.


✅ Trial 1/10 - Accuracy: 0.7203

🔵 Starting Trial 2/10
🔹 Hyperparams - C_svm: 0.032, C_lr: 0.059, alpha_nb: 0.823


C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1744981985.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-2, 1e1)   # More compact range
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1744981985.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-2, 1)       # Keeps it reasonable
[I 2025-04-01 15:07:43,772] Trial 1 finished with value: 0.7214110705535277 and parameters: {'C_svm': 0.03232670980640546, 'C_lr': 0.058892042537674716, 'alpha_nb': 0.8226629452060383}. Best is trial 1 with value: 0.7214110705535277.


...

✅ Trial 8/10 - Accuracy: 0.7160

🔵 Starting Trial 9/10
🔹 Hyperparams - C_svm: 0.116, C_lr: 0.013, alpha_nb: 0.026


C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1744981985.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-2, 1e1)   # More compact range
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1744981985.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-2, 1)       # Keeps it reasonable
[I 2025-04-01 17:41:40,995] Trial 8 finished with value: 0.7221611080554028 and parameters: {'C_svm': 0.11593797647289757, 'C_lr': 0.01263765185388339, 'alpha_nb': 0.025876411200658332}. Best is trial 3 with value: 0.7531376568828441.


✅ Trial 9/10 - Accuracy: 0.7222

🔵 Starting Trial 10/10
🔹 Hyperparams - C_svm: 0.077, C_lr: 0.129, alpha_nb: 0.025


C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1744981985.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-2, 1e1)   # More compact range
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1744981985.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-2, 1)       # Keeps it reasonable
[I 2025-04-01 17:59:06,782] Trial 9 finished with value: 0.7219360968048403 and parameters: {'C_svm': 0.07662861487686683, 'C_lr': 0.12902086073755784, 'alpha_nb': 0.025173072240344086}. Best is trial 3 with value: 0.7531376568828441.


✅ Trial 10/10 - Accuracy: 0.7219

🎯 Best Trial Found:
⭐ Accuracy: 0.7531
🏆 Best Hyperparameters: {'C_svm': 1.8444223425586244, 'C_lr': 0.052603806017407155, 'alpha_nb': 0.15326344811148027}


In [ ]:
# Create and run the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)  # You can increase n_trials for better tuning


[I 2025-03-31 13:26:45,151] A new study created in memory with name: no-name-a9d44c78-8e02-4a20-a1b9-c8d6313dd33e
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use 


🔵 Starting Trial: 1
🔹 Hyperparameters - C_svm: 7.62560, C_lr: 94.49656, alpha_nb: 0.25609


[I 2025-03-31 14:45:03,384] Trial 0 finished with value: 0.7554878040867913 and parameters: {'C_svm': 7.625604008699433, 'C_lr': 94.4965640323167, 'alpha_nb': 0.2560863611859261}. Best is trial 0 with value: 0.7554878040867913.


✅ Trial 1/15 - Accuracy: 0.7555

🔵 Starting Trial: 2
🔹 Hyperparameters - C_svm: 0.07971, C_lr: 0.90715, alpha_nb: 0.00602


C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha_nb = trial.suggest_loguniform("alpha_nb", 1e-3, 1)
[I 2025-03-31 15:

...

✅ Trial 8/15 - Accuracy: 0.7559

🔵 Starting Trial: 9
🔹 Hyperparameters - C_svm: 0.47654, C_lr: 0.01865, alpha_nb: 0.00151


C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha_nb = trial.suggest_loguniform("alpha_nb", 1e-3, 1)
[I 2025-03-31 22:

✅ Trial 9/15 - Accuracy: 0.7459

🔵 Starting Trial: 10
🔹 Hyperparameters - C_svm: 0.00398, C_lr: 69.43612, alpha_nb: 0.35776


C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha_nb = trial.suggest_loguniform("alpha_nb", 1e-3, 1)
[I 2025-03-31 22:

✅ Trial 10/15 - Accuracy: 0.7465


In [ ]:
study.optimize(objective, n_trials=20)


🔵 Starting Trial: 13
🔹 Hyperparameters - C_svm: 413.02871, C_lr: 0.00287, alpha_nb: 0.79532


C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha_nb = trial.suggest_loguniform("alpha_nb", 1e-3, 1)
[I 2025-04-01 00:

✅ Trial 13/15 - Accuracy: 0.7677

🔵 Starting Trial: 14
🔹 Hyperparameters - C_svm: 300.92790, C_lr: 0.00127, alpha_nb: 0.71554


C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha_nb = trial.suggest_loguniform("alpha_nb", 1e-3, 1)
[I 2025-04-01 01:

✅ Trial 14/15 - Accuracy: 0.7673

🔵 Starting Trial: 15
🔹 Hyperparameters - C_svm: 928.02350, C_lr: 0.00278, alpha_nb: 0.64878


...

✅ Trial 24/15 - Accuracy: 0.7669

🔵 Starting Trial: 25
🔹 Hyperparameters - C_svm: 387.44279, C_lr: 0.00823, alpha_nb: 0.47118


C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha_nb = trial.suggest_loguniform("alpha_nb", 1e-3, 1)
[I 2025-04-01 14:

✅ Trial 25/15 - Accuracy: 0.7670

🔵 Starting Trial: 26
🔹 Hyperparameters - C_svm: 292.09296, C_lr: 0.00625, alpha_nb: 0.54449


C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_svm = trial.suggest_loguniform("C_svm", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C_lr = trial.suggest_loguniform("C_lr", 1e-3, 1e3)
C:\Users\rohan\AppData\Local\Temp\ipykernel_19940\1142412665.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha_nb = trial.suggest_loguniform("alpha_nb", 1e-3, 1)
[W 2025-04-01 14:

KeyboardInterrupt: 

In [ ]:
print("\n🎯 Best Parameters Found by Optuna:", study.best_params)


🎯 Best Parameters Found by Optuna: {'C_svm': 1.8444223425586244, 'C_lr': 0.052603806017407155, 'alpha_nb': 0.15326344811148027}


In [ ]:
# Manually set best hyperparameters
C_svm = 413.02871
C_lr = 0.00287
alpha_nb = 0.79532


# Train the final Voting Classifier using the best parameters
final_voting_clf = VotingClassifier(
    estimators=[
        ("svm", SVC(C=413.02871, kernel="rbf", probability=True)),
        ("lr", LogisticRegression(C=0.00287, solver="liblinear", n_jobs=-1)),
        ("nb", MultinomialNB(alpha=0.79532))
    ],
    voting="soft"
)

In [ ]:
# Train the final model
final_voting_clf.fit(X_train_combined, y_train)

C:\Users\rohan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(


VotingClassifier(estimators=[('svm', SVC(C=413.02871, probability=True)),
                             ('lr',
                              LogisticRegression(C=0.00287, n_jobs=-1,
                                                 solver='liblinear')),
                             ('nb', MultinomialNB(alpha=0.79532))],
                 voting='soft')

In [ ]:
# Evaluate accuracy on test set
accuracy = final_voting_clf.score(X_test_combined, y_test)
print("Final Model Accuracy:", accuracy)

# Predict labels for the test set
y_pred = final_voting_clf.predict(X_test_combined)

# Print classification report
print("📊 Classification Report:")
print(classification_report(y_test, y_pred))

Final Model Accuracy: 0.8066
📊 Classification Report:
              precision    recall  f1-score   support

    negative       0.81      0.82      0.82      1706
     neutral       0.76      0.75      0.75      1666
    positive       0.85      0.86      0.85      1628

    accuracy                           0.81      5000
   macro avg       0.81      0.81      0.81      5000
weighted avg       0.81      0.81      0.81      5000

